In [10]:
using ITensors
using Plots
using LinearAlgebra 
using LaTeXStrings


In [2]:
# --- 1. Adjacency Matrix Definitions (Used to define MPO couplings) ---

"""
    generate_linear_matrix(N::Int)

Generates an adjacency matrix for an open 1D chain of N nodes (nearest-neighbor
coupling only, no closed loop/periodic boundary conditions).
"""
function generate_linear_matrix(N::Int)
    A = zeros(N, N)
    for i in 1:N-1
        A[i, i+1] = 1.0
        A[i+1, i] = 1.0
    end
    return A
end

"""
    generate_entangled_matrix(N::Int)

Generates an adjacency matrix for a fully connected (entangled) graph of N nodes
(all nodes coupled to all others).
"""
function generate_entangled_matrix(N::Int)
    A = ones(N, N)
    A -= Matrix{Float64}(I, N, N) # Set diagonal to zero (no self-coupling)
    return A
end

generate_entangled_matrix

In [3]:
# --- 2. Hamiltonian Construction as an MPO ---

"""
    make_XXZ_MPO(sites, J::Float64, Delta::Float64, A::Matrix{Float64})

Constructs the XXZ Hamiltonian as a Matrix Product Operator (MPO) based on
the adjacency matrix A. This uses ITensors' convenient OpSum system.
H = J * sum_{i<j} A_{ij} * (S_i^x S_j^x + S_i^y S_j^y + Delta * S_i^z S_j^z)
"""
function make_XXZ_MPO(sites, J::Float64, Delta::Float64, A::Matrix{Float64})
    N = length(sites)
    os = OpSum()

    # Iterate over all unique pairs (i, j) where i < j
    for i in 1:N
        for j in (i+1):N
            if A[i, j] != 0.0 # Only consider coupled sites
                coeff = J * A[i, j]
                
                # S_i^x S_j^x + S_i^y S_j^y (XX+YY part)
                os += coeff, "Sx", i, "Sx", j
                os += coeff, "Sy", i, "Sy", j

                # Delta * S_i^z S_j^z (ZZ part)
                os += coeff * Delta, "Sz", i, "Sz", j
            end
        end
    end
    
    return MPO(os, sites)
end

make_XXZ_MPO

In [5]:
# --- 3. Entanglement Entropy Calculation from MPS ---

"""
    calculate_bipartite_entropy(psi::MPS, N::Int)

Calculates the Bipartite Entanglement Entropy (Von Neumann Entropy) of the MPS
ground state psi, partitioned exactly at the middle (N/2).

S = -Tr(rho_A log2(rho_A)), derived from the Schmidt coefficients (singular values)
across the partition cut.
"""
function calculate_bipartite_entropy(psi, N::Int)
    L = N ÷ 2 # Midpoint partition

    # Orthogonalize the MPS to the bond L to L+1. This prepares the state
    # such that the singular values of the partition are ready to be extracted.
    orthogonalize!(psi, L) 
    
    # Get the singular values (Schmidt coefficients) from the bond between L and L+1
    # svdvals is a highly optimized function in ITensors for this purpose
    s_vals = svdvals(psi[L])
    
    # Calculate probabilities (Schmidt coefficients squared)
    probabilities = s_vals.^2
    probabilities = probabilities / sum(probabilities) # Normalize (safety check)

    # Calculate Von Neumann Entropy: S = -sum(p * log2(p))
    entropy = 0.0
    for p in probabilities
        if p > 1e-12 # Avoid log(0)
            entropy -= p * log2(p)
        end
    end

    return entropy
end

calculate_bipartite_entropy

In [8]:
# --- 4. Main Simulation and Plotting ---

function run_dmrg_simulation_and_plot(; N_max=10, J=1.0)
    # Define the range of the anisotropy parameter Delta (matches Figure 7 range)
    Delta_range = -2.0:0.05:2.0
    num_deltas = length(Delta_range)

    # Initialize storage for entanglement entropy results
    linear_entropy = zeros(num_deltas)
    entangled_entropy = zeros(num_deltas)

    # --- 4.1 Setup for N = 10 ---
    N = N_max
    
    # Define the physical degrees of freedom (S=1/2 spins)
    sites = siteinds("S=1/2", N)
    
    # Define DMRG parameters (sweeps, max bond dimension, and cutoff)
    sweeps = Sweeps(10) # Number of sweeps (passes) for the DMRG algorithm
    
    # Gradually increase maxdim for better accuracy
    # Fully connected graphs require a higher bond dimension than 1D chains.
    setmaxdim!(sweeps, 10, 20, 40, 60, 100, 150, 200, 300, 300) 
    setcutoff!(sweeps, 1e-10) # Truncation error cutoff

    println("Starting DMRG calculation for N=$N...")

    # --- 4.2 Loop over Delta values ---
    for (k, Delta) in enumerate(Delta_range)
        # --- Linear Graph ---
        A_linear = generate_linear_matrix(N)
        H_linear = make_XXZ_MPO(sites, J, Delta, A_linear)
        
        # Initial random MPS (bond dimension 10 for initialization)
        psi0_linear = randomMPS(sites, 10) 
        
        # Run DMRG
        energy_linear, psi_linear = dmrg(H_linear, psi0_linear, sweeps, outputlevel=0)
        
        # Calculate Entropy
        linear_entropy[k] = calculate_bipartite_entropy(psi_linear, N)

        # --- Entangled (Fully Connected) Graph ---
        A_entangled = generate_entangled_matrix(N)
        # NOTE: Building MPOs for fully connected long-range interactions is much
        # slower and less efficient than for 1D chains, but necessary here.
        H_entangled = make_XXZ_MPO(sites, J, Delta, A_entangled)
        
        psi0_entangled = randomMPS(sites, 10) 
        
        # Run DMRG
        energy_entangled, psi_entangled = dmrg(H_entangled, psi0_entangled, sweeps, outputlevel=0)
        
        # Calculate Entropy
        entangled_entropy[k] = calculate_bipartite_entropy(psi_entangled, N)
        
        print("Completed Delta: $Delta \r")
    end
    println("\nCalculation complete.")

    # --- 4.3 Plotting (Replicate Figure 7) ---

    p = plot(Delta_range, linear_entropy,
             label="Linear Chain (N=$N)",
             linewidth=3,
             linecolor=:blue,
             title="Ground State Entanglement Entropy (DMRG, XXZ Model)",
             xlabel=L"Anisotropy Parameter $\Delta$",
             ylabel=L"Bipartite Entanglement Entropy \$S_L\$ (bits)",
             legend=:topright,
             grid=true,
             minorgrid=true,
             fontfamily="Times",
             dpi=300)

    plot!(p, Delta_range, entangled_entropy,
          label="Entangled/Fully Connected (N=$N)",
          linewidth=3,
          linecolor=:red)

    display(p)

    return p
end

# Execute the simulation and plotting function
run_dmrg_simulation_and_plot(N_max=10)


LoadError: LoadError: UndefVarError: `@L_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at In[8]:68